In [1]:
import numpy as np
from multiple_factor import DGP2, Inferece2
from dgp import DGP
from inference import Inference

def reject_prob(Xdim, num_factor, sample_size, ntrials=1000):
    phi_tau = np.zeros((ntrials,2))
    for i in range(ntrials):
        dgp = DGP2(num_factor, sample_size, Xdim)
        Y, D, tuple_idx = dgp.Y, dgp.D, dgp.tuple_dix
        inf = Inferece2(Y, D, tuple_idx)
        phi_tau[i,0] = inf.phi_tau
        phi_tau[i,1] = inf.phi_tau_p
    return np.mean(phi_tau, axis=0)

"""
def reject_prob1(n, modelY='1', modelDA='8', ate=0, ntrials=1000):
    phi_tau11s, phi_tau10s, phi_theta1s, phi_theta2s, phi_theta12s = np.zeros(ntrials), np.zeros(ntrials), np.zeros(ntrials), np.zeros(ntrials), np.zeros(ntrials)
    for i in range(ntrials):
        dgp = DGP(modelY,modelDA,n,tau=ate)
        inf = Inference(dgp.Y, dgp.D, dgp.A, modelDA, tuple_idx=dgp.tuple_idx, tau=dgp.tau)
        phi_tau11, phi_tau10, phi_theta1, phi_theta2, phi_theta12 = inf.inference()
        phi_tau11s[i] = phi_tau11
        phi_tau10s[i] = phi_tau10
        phi_theta1s[i] = phi_theta1
        phi_theta2s[i] = phi_theta2
        phi_theta12s[i] = phi_theta12
        break
    return phi_tau11s, phi_tau10s, phi_theta1s, phi_theta2s, phi_theta12s
"""


"\ndef reject_prob1(n, modelY='1', modelDA='8', ate=0, ntrials=1000):\n    phi_tau11s, phi_tau10s, phi_theta1s, phi_theta2s, phi_theta12s = np.zeros(ntrials), np.zeros(ntrials), np.zeros(ntrials), np.zeros(ntrials), np.zeros(ntrials)\n    for i in range(ntrials):\n        dgp = DGP(modelY,modelDA,n,tau=ate)\n        inf = Inference(dgp.Y, dgp.D, dgp.A, modelDA, tuple_idx=dgp.tuple_idx, tau=dgp.tau)\n        phi_tau11, phi_tau10, phi_theta1, phi_theta2, phi_theta12 = inf.inference()\n        phi_tau11s[i] = phi_tau11\n        phi_tau10s[i] = phi_tau10\n        phi_theta1s[i] = phi_theta1\n        phi_theta2s[i] = phi_theta2\n        phi_theta12s[i] = phi_theta12\n        break\n    return phi_tau11s, phi_tau10s, phi_theta1s, phi_theta2s, phi_theta12s\n"

In [2]:
res, res_p = np.zeros((4,5)), np.zeros((4,5))
for i, q in enumerate([1, 2, 4, 8]):
    for j, k in enumerate([2, 3, 4, 5, 6]):
        p1, p2 = reject_prob(q, k, 1280, 500)
        res[i,j] = p1
        res_p[i,j] = p2
        print("q={}, k={}, p={},P{}".format(q,k,p1,p2))

np.savetxt("res1.txt", res)
np.savetxt("res2.txt", res_p)
print(res)
print(res_p)

R[write to console]: Notice:
Formerly the gendistance() function scaled the Mahalanobis distances into large
integers, as required by the nonbimatch() function. Starting in version 1.5.0,
gendistance() will return unscaled distances. This facilitates comparison to an
appropriate F distribution for multivariate normal data. Any required scaling
will happen invisibly within nonbimatch(). This notice will be removed in a
future version of nbpMatching.

R[write to console]: 



RRuntimeError: 